## label as normalized order of finish based on number of horses in race
the label should be resricted to a 0 - 1 range taking into account the number of horses per race
so a horse that places 5th in a race of 10 horses gets a score of 0.5

$
\text{label} = 1 - \dfrac{(\text{finish order} - 1)}{(\text{number of horses in race} - 1)}
$

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from IPython.display import display

sys.path.insert(0, '..')
np.random.seed(42)
pd.options.display.max_columns = None
%matplotlib inline

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

/Users/henryehly/src/netkeiba/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/henryehly/src/netkeiba/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [45]:
%%html
<style>
    /* force tables to display at left of cell */
    table {
        display: inline-block
    }
</style>

In [2]:
from scripts.utils import read_netkeiba

df = read_netkeiba()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240471 entries, 0 to 240470
Data columns (total 66 columns):
c_id                                      240471 non-null int64
c_weight_carried                          240471 non-null float64
c_post_position                           240471 non-null int64
c_order_of_finish                         240471 non-null int64
c_order_of_finish_lowered                 240471 non-null int64
c_finish_time                             240471 non-null float64
c_horse_weight                            240471 non-null int64
c_horse_weight_diff                       240471 non-null int64
c_popularity                              240471 non-null int64
c_first_place_odds                        240471 non-null float64
r_id                                      240471 non-null int64
r_key                                     240471 non-null object
r_racetrack                               240471 non-null object
r_course_type                             240471 

In [4]:
df.head()

,c_id,c_weight_carried,c_post_position,c_order_of_finish,c_order_of_finish_lowered,c_finish_time,c_horse_weight,c_horse_weight_diff,c_popularity,c_first_place_odds,r_id,r_key,r_racetrack,r_course_type,r_weather,r_url,r_distance,r_date,r_dirt_condition,r_turf_condition,r_impost_category,r_is_non_winner_regional_horse_allowed,r_is_winner_regional_horse_allowed,r_is_regional_jockey_allowed,r_is_foreign_horse_allowed,r_is_foreign_horse_and_trainer_allowed,r_is_apprentice_jockey_allowed,r_is_female_only,h_id,h_key,h_url,h_total_races,h_total_wins,h_sex,h_birthday,h_user_rating,j_id,j_key,j_url,j_career_1st_place_count,j_career_2nd_place_count,j_career_3rd_place_count,j_career_4th_place_or_below_count,j_career_turf_race_count,j_career_turf_win_count,j_career_dirt_race_count,j_career_dirt_win_count,j_career_1st_place_rate,j_career_1st_2nd_place_rate,j_career_any_place_rate,j_career_earnings,t_id,t_key,t_url,t_career_1st_place_count,t_career_2nd_place_count,t_career_3rd_place_count,t_career_4th_place_or_below_count,t_career_turf_race_count,t_career_turf_win_count,t_career_dirt_race_count,t_career_dirt_win_count,t_career_1st_place_rate,t_career_1st_2nd_place_rate,t_career_any_place_rate,t_career_earnings
0,1,54.0,7,1,0,109.9,514,-2,1,3.7,1,201803030101,fuma,dirt,sunny,http://db.netkeiba.com/race/201803030101/,1700,2018-11-03,good,None,age_based,0,0,0,0,0,0,1,1,2016104408,http://db.netkeiba.com/horse/2016104408/,2,1,female,2016-04-03,3.17,1,01092,http://db.netkeiba.com/jockey/result/01092,423,477,565,6501,4156,222,3810,201,0.053,0.113,0.184,829574.0,1,00359,http://db.netkeiba.com/trainer/result/00359,556,577,557,5185,3613,299,3052,241,0.081,0.165,0.246,892578.6
1,2,54.0,8,2,0,110.1,490,0,3,6.3,1,201803030101,fuma,dirt,sunny,http://db.netkeiba.com/race/201803030101/,1700,2018-11-03,good,None,age_based,0,0,0,0,0,0,1,2,2016100908,http://db.netkeiba.com/horse/2016100908/,3,0,female,2016-02-07,3.25,2,01109,http://db.netkeiba.com/jockey/result/01109,142,182,189,3746,2091,74,2167,68,0.033,0.076,0.120,253625.0,2,01147,http://db.netkeiba.com/trainer/result/01147,92,84,81,866,508,36,537,45,0.082,0.157,0.229,136755.2
2,3,54.0,4,3,0,110.4,464,10,4,6.4,1,201803030101,fuma,dirt,sunny,http://db.netkeiba.com/race/201803030101/,1700,2018-11-03,good,None,age_based,0,0,0,0,0,0,1,3,2016105892,http://db.netkeiba.com/horse/2016105892/,2,0,female,2016-04-29,NaN,3,01128,http://db.netkeiba.com/jockey/result/01128,330,299,350,4005,2498,146,2486,184,0.066,0.126,0.196,521757.2,3,01006,http://db.netkeiba.com/trainer/result/01006,286,255,279,3530,1854,111,2237,154,0.066,0.124,0.189,454950.2
3,4,54.0,6,4,0,110.4,450,-6,2,4.5,1,201803030101,fuma,dirt,sunny,http://db.netkeiba.com/race/201803030101/,1700,2018-11-03,good,None,age_based,0,0,0,0,0,0,1,4,2016106313,http://db.netkeiba.com/horse/2016106313/,3,0,female,2016-03-19,NaN,4,01015,http://db.netkeiba.com/jockey/result/01015,461,528,542,7051,4271,197,4311,264,0.054,0.115,0.178,852023.5,4,01099,http://db.netkeiba.com/trainer/result/01099,269,246,289,2686,1831,136,1558,118,0.077,0.148,0.230,493068.1
4,5,54.0,6,5,0,110.5,426,-8,10,43.3,1,201803030101,fuma,dirt,sunny,http://db.netkeiba.com/race/201803030101/,1700,2018-11-03,good,None,age_based,0,0,0,0,0,0,1,5,2016102321,http://db.netkeiba.com/horse/2016102321/,2,0,female,2016-05-21,NaN,5,01117,http://db.netkeiba.com/jockey/result/01117,372,429,435,5399,3408,201,3227,171,0.056,0.121,0.186,660899.9,5,01131,http://db.netkeiba.com/trainer/result/01131,60,71,77,1541,703,16,1008,43,0.034,0.075,0.119,94999.5


In [5]:
df['r_contender_count'] = df.groupby('r_id').c_id.count().loc[df.r_id].values
df['c_norm_order_of_finish'] = 1.0 - (df.c_order_of_finish - 1) / (df.r_contender_count - 1)

index_cols = ['c_id', 'r_id', 'h_id', 'j_id', 't_id', 'r_key', 'r_url', 'h_key', 'h_url', 'j_key', 'j_url', 't_key', 't_url']
label_cols = ['c_norm_order_of_finish', 'c_order_of_finish', 'c_finish_time', 'c_order_of_finish_lowered']

X = df.drop(columns=index_cols + label_cols)
y = df.c_norm_order_of_finish

X.shape, y.shape

((240471, 51), (240471,))

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((192376, 51), (192376,), (48095, 51), (48095,))

In [7]:
# fillna('')
# OneHotEncoder(handle_unknown='ignore')
cat_attribs = [
    'r_racetrack', 'r_course_type', 'r_weather', 'h_sex', 
    'r_impost_category', 'r_dirt_condition', 'r_turf_condition'
]

# Use as-is
bool_attribs = [
    'r_is_non_winner_regional_horse_allowed', 'r_is_winner_regional_horse_allowed',
    'r_is_regional_jockey_allowed', 'r_is_foreign_horse_allowed',
    'r_is_foreign_horse_and_trainer_allowed', 'r_is_apprentice_jockey_allowed',
    'r_is_female_only'
]

# CombinedDateAttributesAdder
# Imputer(strategy='median')
# StandardScaler
date_attribs = [
    'r_date', 'h_birthday'
]

# Imputer(strategy='median')
# CombinedNumericAttributesAdder
# StandardScaler
num_attribs = [
    'c_weight_carried', 'c_post_position',
    'c_horse_weight', 'c_horse_weight_diff', 'c_popularity',
    'c_first_place_odds', 'r_distance', 'r_contender_count',
    'h_total_races', 'h_total_wins', 'h_user_rating',
    'j_career_1st_place_count', 'j_career_2nd_place_count',
    'j_career_3rd_place_count', 'j_career_4th_place_or_below_count',
    'j_career_turf_race_count', 'j_career_turf_win_count',
    'j_career_dirt_race_count', 'j_career_dirt_win_count',
    'j_career_1st_place_rate', 'j_career_1st_2nd_place_rate',
    'j_career_any_place_rate', 'j_career_earnings',
    't_career_1st_place_count', 't_career_2nd_place_count',
    't_career_3rd_place_count', 't_career_4th_place_or_below_count',
    't_career_turf_race_count', 't_career_turf_win_count',
    't_career_dirt_race_count', 't_career_dirt_win_count',
    't_career_1st_place_rate', 't_career_1st_2nd_place_rate',
    't_career_any_place_rate', 't_career_earnings'
]

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin


class CombinedDateAttributesAdder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df = pd.DataFrame(X)
        df['h_age_days'] = (pd.to_datetime(df['r_date']) - pd.to_datetime(df['h_birthday'])).dt.days
        return df.drop(columns=['r_date', 'h_birthday']).values

    
class CombinedNumericAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        # passing 'num_attribs' via init is preferable, but 'not found' errors occurs in pipeline
        self.j_turf_wins_ix = num_attribs.index('j_career_turf_win_count')
        self.j_turf_races_ix = num_attribs.index('j_career_turf_race_count')
        self.j_dirt_wins_ix = num_attribs.index('j_career_dirt_win_count')
        self.j_dirt_races_ix = num_attribs.index('j_career_dirt_race_count')
        self.t_turf_wins_ix = num_attribs.index('t_career_turf_win_count')
        self.t_turf_races_ix = num_attribs.index('t_career_turf_race_count')
        self.t_dirt_wins_ix = num_attribs.index('t_career_dirt_win_count')
        self.t_dirt_races_ix = num_attribs.index('t_career_dirt_race_count')
        self.h_total_wins_ix = num_attribs.index('h_total_wins')
        self.h_total_races_ix = num_attribs.index('h_total_races')
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        j_turf_wins, j_turf_races = X[:, self.j_turf_wins_ix], X[:, self.j_turf_races_ix]
        j_turf_win_rate = np.divide(j_turf_wins, j_turf_races, out=np.zeros_like(j_turf_wins), where=j_turf_wins!=0.)

        j_dirt_wins, j_dirt_races = X[:, self.j_dirt_wins_ix], X[:, self.j_dirt_races_ix]
        j_dirt_win_rate = np.divide(j_dirt_wins, j_dirt_races, out=np.zeros_like(j_dirt_wins), where=j_dirt_wins!=0.)

        t_turf_wins, t_turf_races = X[:, self.t_turf_wins_ix], X[:, self.t_turf_races_ix]
        t_turf_win_rate = np.divide(t_turf_wins, t_turf_races, out=np.zeros_like(t_turf_wins), where=t_turf_wins!=0.)

        t_dirt_wins, t_dirt_races = X[:, self.t_dirt_wins_ix], X[:, self.t_dirt_races_ix]
        t_dirt_win_rate = np.divide(t_dirt_wins, t_dirt_races, out=np.zeros_like(t_dirt_wins), where=t_dirt_wins!=0.)

        h_wins, h_races = X[:, self.h_total_wins_ix], X[:, self.h_total_races_ix]
        h_win_rate = np.divide(h_wins, h_races, out=np.zeros_like(h_wins), where=h_wins!=0.)

        return np.c_[X, j_turf_win_rate, j_dirt_win_rate, t_turf_win_rate, t_dirt_win_rate, h_win_rate]

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Numeric Attributes

In [10]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('num_attribs_adder', CombinedNumericAttributesAdder()),
    ('std_scaler', StandardScaler())
])

added_columns = ['j_career_turf_win_rate', 'j_career_dirt_win_rate', 
                 't_career_turf_win_rate', 't_career_dirt_win_rate', 'h_win_rate']

X_train_num = num_pipeline.fit_transform(X_train[num_attribs])
X_train_num_df = pd.DataFrame(X_train_num, columns=num_attribs+added_columns)
X_train_num_df.describe()

,c_weight_carried,c_post_position,c_horse_weight,c_horse_weight_diff,c_popularity,c_first_place_odds,r_distance,r_contender_count,h_total_races,h_total_wins,h_user_rating,j_career_1st_place_count,j_career_2nd_place_count,j_career_3rd_place_count,j_career_4th_place_or_below_count,j_career_turf_race_count,j_career_turf_win_count,j_career_dirt_race_count,j_career_dirt_win_count,j_career_1st_place_rate,j_career_1st_2nd_place_rate,j_career_any_place_rate,j_career_earnings,t_career_1st_place_count,t_career_2nd_place_count,t_career_3rd_place_count,t_career_4th_place_or_below_count,t_career_turf_race_count,t_career_turf_win_count,t_career_dirt_race_count,t_career_dirt_win_count,t_career_1st_place_rate,t_career_1st_2nd_place_rate,t_career_any_place_rate,t_career_earnings,j_career_turf_win_rate,j_career_dirt_win_rate,t_career_turf_win_rate,t_career_dirt_win_rate,h_win_rate
count,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05,1.923760e+05
mean,1.300319e-15,-6.419321e-17,5.915526e-16,-2.452491e-17,4.387890e-17,-9.924462e-17,1.483683e-16,-5.917003e-17,-5.861601e-17,2.755359e-17,5.594929e-16,2.193945e-17,-1.861529e-17,-2.031431e-17,-1.082383e-16,-6.796059e-17,-2.777520e-17,4.967771e-18,4.284472e-17,3.560544e-17,4.103490e-16,3.338933e-17,6.426708e-17,9.036173e-17,-5.844980e-18,1.042678e-16,6.914251e-17,6.581835e-17,-2.519897e-17,-3.763687e-17,1.578976e-16,-3.618163e-16,-1.243236e-16,-6.644994e-16,-1.502889e-16,4.299246e-17,-3.892960e-17,1.300300e-16,2.852867e-16,-3.527302e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-4.500140e+00,-1.647437e+00,-4.457277e+00,-7.092334e+00,-1.544255e+00,-6.894195e-01,-3.924732e+00,-4.172835e+00,-1.446184e+00,-1.102005e+00,-3.290279e+00,-9.327761e-01,-1.050533e+00,-1.144817e+00,-1.500234e+00,-1.333858e+00,-8.825783e-01,-1.441293e+00,-9.640961e-01,-1.812043e+00,-2.141611e+00,-2.476578e+00,-8.896405e-01,-1.464245e+00,-1.635323e+00,-1.752141e+00,-2.105824e+00,-1.735867e+00,-1.154751e+00,-2.129329e+00,-1.672020e+00,-2.873765e+00,-3.480223e+00,-4.049515e+00,-1.254713e+00,-1.602492e+00,-1.903641e+00,-2.505775e+00,-2.932350e+00,-1.149429e+00
25%,-5.628271e-01,-7.676061e-01,-6.780781e-01,-6.696089e-01,-8.718885e-01,-6.090987e-01,-8.259402e-01,-7.419648e-01,-7.398654e-01,-6.838309e-01,2.593818e-01,-7.509091e-01,-8.191047e-01,-8.587348e-01,-9.443246e-01,-9.474602e-01,-7.510709e-01,-8.901905e-01,-7.640513e-01,-7.004926e-01,-7.162134e-01,-7.395057e-01,-7.352807e-01,-7.181309e-01,-7.618059e-01,-8.010976e-01,-7.312749e-01,-7.482462e-01,-6.854016e-01,-7.623145e-01,-7.903872e-01,-6.305861e-01,-6.492817e-01,-6.397179e-01,-7.062319e-01,-6.641036e-01,-7.577131e-01,-7.397111e-01,-6.340793e-01,-6.543274e-01
50%,-3.537825e-04,1.122244e-01,-1.506071e-02,-4.300157e-02,2.460060e-02,-4.310904e-01,-1.097327e-01,5.446116e-01,-1.512662e-01,-2.656565e-01,2.593818e-01,-3.032367e-01,-1.900098e-01,-1.699514e-01,-2.067953e-02,-5.066788e-02,-2.960552e-01,-7.740826e-02,-2.580557e-01,-1.952423e-01,-2.510836e-01,-1.952230e-01,-2.823603e-01,-2.253271e-01,-2.303284e-01,-1.440131e-01,-1.009607e-01,-1.268608e-01,-3.187221e-

# DateTime Attributes

In [11]:
X_train[date_attribs].describe()

,r_date,h_birthday
count,192376,192376
unique,535,1660
top,2015-05-23,2012-04-30
freq,461,643


In [12]:
date_pipeline = Pipeline([
    ('date_attribs_adder', CombinedDateAttributesAdder()),
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

pd.DataFrame(date_pipeline.fit_transform(X_train[date_attribs])).describe()

,0
count,1.923760e+05
mean,-1.528375e-16
std,1.000003e+00
min,-1.424788e+00
25%,-7.349290e-01
50%,-3.597788e-01
75%,5.676759e-01
max,6.069879e+00


# Boolean Attributes

In [13]:
X_train[bool_attribs].astype(bool).describe()

,r_is_non_winner_regional_horse_allowed,r_is_winner_regional_horse_allowed,r_is_regional_jockey_allowed,r_is_foreign_horse_allowed,r_is_foreign_horse_and_trainer_allowed,r_is_apprentice_jockey_allowed,r_is_female_only
count,192376,192376,192376,192376,192376,192376,192376
unique,1,2,1,1,2,2,2
top,False,False,False,False,False,False,False
freq,192376,171472,192376,192376,184494,190652,162794


# Categorical Attributes

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
X.r_dirt_condition.value_counts(dropna=False)

NaN               120464
good               70944
slightly_heavy     25888
heavy              15379
bad                 7796
Name: r_dirt_condition, dtype: int64

In [16]:
X.r_turf_condition.value_counts(dropna=False)

NaN               117982
good               98109
slightly_heavy     17023
heavy               6211
bad                 1146
Name: r_turf_condition, dtype: int64

In [17]:
one_hot_enc = OneHotEncoder(
    sparse=False, 
    categories=[['bad', 'good', 'heavy', 'slightly_heavy']],
    handle_unknown='ignore')
one_hot_enc.fit_transform(X_train[['r_turf_condition']]), one_hot_enc.categories_

(array([[0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 1., 0., 0.]]),
 [array(['bad', 'good', 'heavy', 'slightly_heavy'], dtype=object)])

In [18]:
cat_attrib_categories = {
    'r_racetrack': ['chukyo', 'fuma', 'hakodate', 'hanshin', 'kyoto', 'nakayama', 'niigata', 'ogura', 'sapporo', 'tokyo'],
    'r_course_type': ['dirt', 'obstacle', 'turf'],
    'r_weather': ['cloudy', 'rainy', 'snowy', 'sunny'],
    'h_sex': ['castrated', 'female', 'male'],
    'r_impost_category': ['age_based', 'age_sex_based', 'decided_per_race', 'handicap'],
    'r_dirt_condition': ['bad', 'good', 'heavy', 'slightly_heavy'],
    'r_turf_condition': ['bad', 'good', 'heavy', 'slightly_heavy']
}

cat_attribs = list(cat_attrib_categories.keys())
categories = list(cat_attrib_categories.values())

cat_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore', categories=categories)
cat_encoder.fit_transform(X_train[cat_attribs])

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [19]:
cat_encoder.categories_

[array(['chukyo', 'fuma', 'hakodate', 'hanshin', 'kyoto', 'nakayama',
        'niigata', 'ogura', 'sapporo', 'tokyo'], dtype=object),
 array(['dirt', 'obstacle', 'turf'], dtype=object),
 array(['cloudy', 'rainy', 'snowy', 'sunny'], dtype=object),
 array(['castrated', 'female', 'male'], dtype=object),
 array(['age_based', 'age_sex_based', 'decided_per_race', 'handicap'],
       dtype=object),
 array(['bad', 'good', 'heavy', 'slightly_heavy'], dtype=object),
 array(['bad', 'good', 'heavy', 'slightly_heavy'], dtype=object)]

In [20]:
cat_pipeline = Pipeline([
    ('one_hot', OneHotEncoder(sparse=False, handle_unknown='ignore', categories=categories))
])

# Full Pipeline

In [21]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs),
    ('date', date_pipeline, date_attribs)
])

full_pipeline.fit_transform(X_train)

array([[-0.56282706,  0.1122244 ,  0.31644799, ...,  0.        ,
         1.        , -1.15384668],
       [-0.56282706,  0.1122244 , -0.61177637, ...,  0.        ,
         0.        , -0.41188295],
       [-1.12530035,  0.55213968, -0.34656941, ...,  0.        ,
         0.        ,  0.09665398],
       ...,
       [ 2.81201263,  1.43197022, -0.81068159, ...,  0.        ,
         0.        ,  1.63268564],
       [ 0.5621195 , -0.32769087,  0.78056017, ...,  0.        ,
         0.        ,  0.75733517],
       [ 1.12459278, -0.32769087, -0.14766419, ...,  0.        ,
         0.        ,  1.67436899]])

# Model Selection

In [22]:
X_train_prepared = full_pipeline.fit_transform(X_train)

### LinearRegression

In [23]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression(n_jobs=-1)
lin_reg.fit(X_train_prepared, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [24]:
X_train_sample = full_pipeline.fit_transform(X_train[:10])
y_train_sample = y_train[:10]

sample_pred = lin_reg.predict(X_train_sample)
pred_vs_actual = np.c_[sample_pred, y_train_sample.values]
pd.DataFrame(np.c_[pred_vs_actual, sample_pred - y_train_sample.values], columns=['prediction', 'label', 'diff'])

,prediction,label,diff
0,0.643799,0.800000,-0.156201
1,0.510010,0.357143,0.152867
2,0.366211,0.100000,0.266211
3,0.359375,0.466667,-0.107292
4,0.285278,0.200000,0.085278
5,0.626831,0.900000,-0.273169
6,0.604614,0.285714,0.318900
7,0.647339,0.250000,0.397339
8,0.688721,0.733333,-0.044613
9,0.358276,0.533333,-0.175057


In [25]:
from sklearn.metrics import mean_squared_error

lin_reg_pred = lin_reg.predict(X_train_prepared)
lin_mse = mean_squared_error(y_train, lin_reg_pred)

np.sqrt(lin_mse)

0.25601567709799133

### DecisionTreeRegressor

In [26]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train_prepared, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [27]:
tree_predictions = tree_reg.predict(X_train_prepared)
tree_mse = mean_squared_error(y_train, tree_predictions)

np.sqrt(tree_mse)

1.2879864487311744e-17

### RandomForestRegressor

In [28]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(X_train_prepared, y_train)

/Users/henryehly/src/netkeiba/venv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [29]:
forest_predictions = forest_reg.predict(X=X_train_prepared)
forest_mse = mean_squared_error(y_true=y_train, y_pred=forest_predictions)

np.sqrt(forest_mse)

0.11138481808170068

### LinearSVR

In [30]:
from sklearn.svm import LinearSVR

lin_svreg = LinearSVR(random_state=42, verbose=5)
lin_svreg.fit(X_train_prepared, y_train)

[LibLinear]

/Users/henryehly/src/netkeiba/venv/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=42, tol=0.0001, verbose=5)

In [33]:
lin_svreg_predictions = lin_svreg.predict(X_train_prepared)
lin_svreg_mse = mean_squared_error(y_true=y_train, y_pred=lin_svreg_predictions)

np.sqrt(lin_svreg_mse)

0.2618646663196695

### SGDRegressor
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor

In [35]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(random_state=42)
sgd_reg.fit(X_train_prepared, y_train)

/Users/henryehly/src/netkeiba/venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=42, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [36]:
sgd_predictions = sgd_reg.predict(X_train_prepared)
sgd_mse = mean_squared_error(y_true=y_train, y_pred=sgd_predictions)

np.sqrt(sgd_mse)

0.25915695591579146

### TheilSenRegressor
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.TheilSenRegressor.html#sklearn.linear_model.TheilSenRegressor

In [37]:
from sklearn.linear_model import TheilSenRegressor

ts_reg = TheilSenRegressor(random_state=42)
ts_reg.fit(X_train_prepared, y_train)

TheilSenRegressor(copy_X=True, fit_intercept=True, max_iter=300,
         max_subpopulation=10000, n_jobs=None, n_subsamples=None,
         random_state=42, tol=0.001, verbose=False)

In [38]:
ts_predictions = ts_reg.predict(X_train_prepared)
ts_mse = mean_squared_error(y_true=y_train, y_pred=ts_predictions)

np.sqrt(ts_mse)

0.25632188340399753

## Quick n' dirty model performance comparison

| model | rmse
| :- | :-
| LinearRegression | 0.256
| DecisionTreeRegressor | 1.288e-17
| RandomForestRegressor | 0.111
| LinearSVR | 0.262
| SGDRegressor | 0.259
| TheilSenRegressor | 0.256